# Data Preparation

In this notebook, I am going to prepare and clean up the data before performing my analysis. Typically, I would write nearly all of this code into a text file, but here I've shown as much of my work as possible. For larger code blocks, I've written functions that you can find in **LINK**. 

I've sourced .csv's from a few places:


* [**Lahman’s Baseball Database**](http://www.seanlahman.com/baseball-archive/statistics/): Sean Lahman has compiled all sorts of baseball data on his website, and I chose to copy the exact 2018 download to the directory [./baseballdatabank-2019.2/](./baseballdatabank-2019.2/).


* [**Baseball Reference's Free Agent Pages**](https://www.baseball-reference.com/leagues/MLB/2016-free-agents.shtml): Each year, Baseball Refence tracks players granted free agency or released between October and December. If a  player is signed within 6 months of becoming a free agent in a given year, they are included in the "Free Agent Signings" table. Otherwise, they are included in the "Free Agent Batters" table. I have downloaded these two tables back to 2006 and saved them to [./free_agents/](./free_agents/). The "Free Agent Signings" tables are saved to [./free_agents/signed/](./free_agents/signed/), and the "Free Agent Batters" tables are saved to [./free_agents/unsigned/](./free_agents/unsigned/).


* [**Fangraphs**](https://www.fangraphs.com): I've chosen wRC+ as my dependent variable (don't worry, I explain the statistic below). That calculation requires a few different sets of player-indepenent data:

    * [wOBA Constants](https://www.fangraphs.com/guts.aspx?type=cn) includes weights needed for each type of hit in a given year. I've saved this file as [wOBA_constants.csv](./constants/)
    
     * League Adjustment for [NL]() and [AL]() includes league-wide stats for both the National and American Leagues. Unfornunately, Fangraphs requires that each year be downloaded individually, so I have saved each year's csv to [./league_adj/nl/](./league_adj/nl/) for the National League and [./league_adj/al/](./league_adj/al/) for the American League.
      
    * [Park Factors](https://www.fangraphs.com/guts.aspx?type=pf&teamid=0&season=2018) includes relative difficulty of hitting each type of hit (e.g. single, double, etc.) in each major league stadium on a per year basis. I downloaded each year's park factors to a csv. The files are located at [./park_factors/](./park_factors/).
    
    
* [**Cot's Baseball Contracts**](https://legacy.baseballprospectus.com/compensation/cots/): Cot's at Baseball Prospectus tracks major league service time, as defined in the MLB's labor agreement. Since the offical calculation uses the player's number of days on the roster, I had to find a data source that tracks service time specifically. These files are located at [./service/time](./service/time).

In [1]:
import os, re
import numpy as np
import pandas as pd

%run functions.py

First, let's import the main batting statistics dataframe and see what it contains.

In [2]:
batting_df_full = pd.read_csv(FILE_LOCS['batting'])
print(batting_df_full.columns.values)

['playerID' 'yearID' 'stint' 'teamID' 'lgID' 'G' 'AB' 'R' 'H' '2B' '3B'
 'HR' 'RBI' 'SB' 'CS' 'BB' 'SO' 'IBB' 'HBP' 'SH' 'SF' 'GIDP']


In [3]:
batting_df_full.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


Immediately, we have a two problems here:
1. We have over 100 years of data we don't need!
2. What should we use as our dataframe index?

I am going to tackle item #1 first, and filter out data before 2006. I've chosen 2006 here since it's the first year of the 2006-2011 labor agreement.

In [4]:
START_YEAR = 2006

batting_df = batting_df_full[batting_df_full['yearID']>=START_YEAR]
batting_df['yearID'].min()

2006

Ok, now for item 2. My first thought was to use the 'playerID' column as the index, but if we do that, we'll have repeating indexes when players play for more than one season.

A better strategy would be to use "player-season", as defined by the columns "playerID" and "yearID". I wrote a handy function to concatinate these columns itemwise.

In [5]:
batting_df['player_season'] = get_combined_index(batting_df, 'playerID', 
                                   'yearID', right_is_num=True)
batting_df = batting_df.set_index(batting_df['player_season'])
batting_df.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,player_season
player_season,,,,,,,,,,,,,,,,,,,,,
aardsda01_2006,aardsda01,2006,1,CHN,NL,45,2,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,aardsda01_2006
abadan01_2006,abadan01,2006,1,CIN,NL,5,3,0,0,0,...,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,abadan01_2006
abercre01_2006,abercre01,2006,1,FLO,NL,111,255,39,54,12,...,6.0,5.0,18,78.0,2.0,3.0,4.0,1.0,2.0,abercre01_2006
abreubo01_2006,abreubo01,2006,1,PHI,NL,98,339,61,94,25,...,20.0,4.0,91,86.0,5.0,2.0,0.0,6.0,8.0,abreubo01_2006
abreubo01_2006,abreubo01,2006,2,NYA,AL,58,209,37,69,16,...,10.0,2.0,33,52.0,1.0,1.0,2.0,3.0,5.0,abreubo01_2006


Next, I suspect we will have pitchers in our batting stats table. If so, we will have to filter out the pitchers. Let's check: is my favorite pitcher, Jacob deGrom in the dataframe?

In [6]:
batting_df[batting_df['playerID']=='degroja01']

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,player_season
player_season,,,,,,,,,,,,,,,,,,,,,
degroja01_2014,degroja01,2014,1,NYN,NL,23,46,3,10,2,...,0.0,0.0,1,13.0,0.0,0.0,6.0,0.0,2.0,degroja01_2014
degroja01_2015,degroja01,2015,1,NYN,NL,31,59,3,11,1,...,0.0,0.0,3,16.0,0.0,0.0,4.0,0.0,4.0,degroja01_2015
degroja01_2016,degroja01,2016,1,NYN,NL,27,42,6,6,1,...,0.0,0.0,3,12.0,0.0,0.0,5.0,0.0,1.0,degroja01_2016
degroja01_2017,degroja01,2017,1,NYN,NL,37,71,6,15,1,...,1.0,0.0,2,22.0,0.0,0.0,4.0,0.0,0.0,degroja01_2017
degroja01_2018,degroja01,2018,1,NYN,NL,32,67,1,11,1,...,0.0,0.0,4,25.0,0.0,0.0,3.0,0.0,1.0,degroja01_2018


Looks like he is! While deGrom is a pretty good batter for a pitcher, pitcher free agents are not rewarded for their hitting abilities, so we should filter them out.

Luckily, there's a pitching stats csv we can use. My approach here might be simplistic since position players sometimes pitch in a blow out, but high value free agents are unlikely to be put in that position.

In [7]:
pitching_df = pd.read_csv(FILE_LOCS['pitching'])
pitching_df = pitching_df[pitching_df['yearID']>=START_YEAR]
pitching_df = pitching_df.set_index(get_combined_index(pitching_df, 
                                                       'playerID', 'yearID',
                                                      right_is_num=True))

batting_df = batting_df.loc[set(batting_df.index.values) - set(pitching_df.index.values)]
batting_df[batting_df['playerID']=='degroja01']

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,player_season
player_season,,,,,,,,,,,,,,,,,,,,,


Great, now all the pitchers have been removed from our batting stats table.

For this project, I've chosen to use wRC+ as my independent variable. You can read about in it depth [here](https://library.fangraphs.com/offense/wrc/), but my reasoning is as follows: players play in different stadiums, leagues, and historical "run environments". For example, hitters on the Colorado Rockies get the unique advantage of playing at high altitudes, making it easier for them to hit home runs.

Even more importantly, batters playing in different time periods are affected by the style of play. While baseball stats are clearly different when comparing 1871 to 2019, the same differences hold true only a few years apart. Even the most casual fan knows about the MLB's [steroid era](https://en.wikipedia.org/wiki/Doping_in_baseball), but [over the last few years, home runs have been increasing](https://www.theringer.com/2017/6/14/16044264/2017-mlb-home-run-spike-juiced-ball-testing-reveal-155cd21108bc).

wRC+ seeks to eliminate these biases as much as possible by adjusting for home ball park, league, and year. It's designed so the average MLB player has a wRC+ of 100. A player with a wRC+ of 110 created 10% more runs then average, and a player with a wRC+ of 90 created 10% less.

I'll read in the csv's for our wRC+ calcuation.

In [8]:
wOBA_constants_df = pd.read_csv(FILE_LOCS['wOBA'], index_col=0)
wOBA_constants_df.head()

,wOBA,wOBAScale,wBB,wHBP,w1B,w2B,w3B,wHR,runSB,runCS,R/PA,R/W,cFIP
Season,,,,,,,,,,,,,
2019,0.321,1.152,0.691,0.720,0.869,1.215,1.526,1.934,0.2,-0.438,0.127,10.341,3.211
2018,0.315,1.226,0.690,0.720,0.880,1.247,1.578,2.031,0.2,-0.407,0.117,9.714,3.161
2017,0.321,1.185,0.693,0.723,0.877,1.232,1.552,1.980,0.2,-0.423,0.122,10.048,3.158
2016,0.318,1.212,0.691,0.721,0.878,1.242,1.569,2.015,0.2,-0.410,0.118,9.778,3.147
2015,0.313,1.251,0.687,0.718,0.881,1.256,1.594,2.065,0.2,-0.392,0.113,9.421,3.134


In [9]:
lg_names = ['NL', 'AL']
lg_dfs = [get_df_from_dir(DIRECTORIES[lg]) for lg in lg_names]
formated_lg_dfs = [format_lg_df(lg_df, lg_name) 
                   for lg_df, lg_name in zip(lg_dfs, lg_names)]
lg_factors_df = pd.concat(formated_lg_dfs)
lg_factors_df.head()

,Season,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,ISO,Spd,BABIP,UBR,wGDP,wSB,wRC,wRAA,wOBA,wRC+,league
2006_NL,2006,94664,8.9 %,16.4 %,0.54,0.272,0.343,0.442,0.785,0.170,4.8,0.302,33.8,5.7,1.5,12380,500.1,0.338,100,NL
2007_NL,2007,95178,8.8 %,16.4 %,0.54,0.273,0.342,0.436,0.778,0.163,4.6,0.304,-1.0,-1.2,17.5,12359,585.3,0.338,100,NL
2018_NL,2018,88080,9.0 %,21.6 %,0.42,0.254,0.327,0.417,0.744,0.163,4.4,0.301,33.0,8.3,-9.3,10757,460.8,0.321,100,NL
2010_NL,2010,93061,8.8 %,18.5 %,0.48,0.262,0.332,0.413,0.745,0.151,4.4,0.302,29.2,8.2,-16.8,11155,462.2,0.327,100,NL
2009_NL,2009,93840,9.3 %,17.5 %,0.53,0.266,0.339,0.422,0.761,0.156,4.7,0.302,16.1,-4.6,-21.9,11543,300.6,0.333,100,NL


In [10]:
park_factors_df = get_df_from_dir(DIRECTORIES['park_factors'])
park_factors_df.head()

,Season,Team,Basic,1B,2B,3B,HR,SO,BB,GB,FB,LD,IFFB,FIP
0,2008,Angels,99,102,99,86,97,99,98,99,102,101,95,99
1,2008,Orioles,102,104,98,91,110,98,98,102,101,101,97,104
2,2008,Red Sox,104,100,115,96,94,98,99,101,100,102,100,97
3,2008,White Sox,104,99,99,84,113,101,104,99,101,99,102,105
4,2008,Indians,99,100,100,89,94,101,102,101,97,102,100,98


Our wOBA_constants_df looks great right out of the csv. For lg_factors_df, I wrote a little function that creates a "season-league" index that uses the same function we used for  "player-season" under the hood.

However, park_factors_df is a little trickier. A "season-team" index would make sense, but we would need a way to match entire team names.

Lahman's data includes a teams.csv. I will import it and see what it contains.

In [11]:
teams_df = pd.read_csv(FILE_LOCS['teams'], index_col=0)
teams_df = teams_df.drop('NAassoc', axis=1)
teams_df.head()

,franchName,active
franchID,,
ALT,Altoona Mountain City,N
ANA,Los Angeles Angels of Anaheim,Y
ARI,Arizona Diamondbacks,Y
ATH,Philadelphia Athletics,N
ATL,Atlanta Braves,Y


"Altoona Mountain City"?! I will filter this dataframe by active teams.

In [12]:
active_teams_df = teams_df[teams_df['active']=='Y']
active_teams_df.head()

,franchName,active
franchID,,
ANA,Los Angeles Angels of Anaheim,Y
ARI,Arizona Diamondbacks,Y
ATL,Atlanta Braves,Y
BAL,Baltimore Orioles,Y
BOS,Boston Red Sox,Y


Better, but do our team abbreviations match our batter_df?

In [13]:
batting_teams = set(batting_df['teamID'])
active_teams = set(active_teams_df.index.values)

print(batting_teams ^ active_teams)

{'TBA', 'ANA', 'NYN', 'CHW', 'SDP', 'KCR', 'SFG', 'FLO', 'CHC', 'LAD', 'CHA', 'LAN', 'STL', 'CHN', 'SFN', 'FLA', 'WSN', 'LAA', 'MIA', 'KCA', 'SLN', 'TBD', 'NYM', 'NYY', 'WAS', 'NYA', 'SDN'}


They do not, but we can quickly make a mapping table by hand.

In [14]:
missing_batting_teams = batting_teams - active_teams
missing_active_teams = active_teams - batting_teams
print(missing_batting_teams)
print(missing_active_teams)

{'FLO', 'CHN', 'TBA', 'SFN', 'NYN', 'WAS', 'CHA', 'LAA', 'MIA', 'NYA', 'LAN', 'KCA', 'SLN', 'SDN'}
{'FLA', 'WSN', 'ANA', 'CHC', 'NYM', 'LAD', 'NYY', 'CHW', 'STL', 'SDP', 'KCR', 'TBD', 'SFG'}


In [15]:
matched_teams = list(batting_teams.intersection(active_teams))
act_team_ids = ['CHW', 'CHC', 'ANA', 'LAD', 'NYY', 'NYM',
                'STL', 'SFG', 'KCR', 'SDP', 'FLA', 'TBD',
                'WSN','FLA'] + matched_teams
batting_ids = ['CHA', 'CHN', 'LAA', 'LAN', 'NYA', 'NYN',
               'SLN', 'SFN', 'KCA', 'SDN', 'MIA', 'TBA',
               'WAS', 'FLO'] + matched_teams
TEAM_ABV_DICT_FOR_REMAP = {'at_id': act_team_ids,
                            'batting_id': batting_ids}


abv_mapping_df = pd.DataFrame(TEAM_ABV_DICT_FOR_REMAP)
abv_mapping_df = abv_mapping_df.set_index(abv_mapping_df['batting_id'])
abv_mapping_df.head()

,at_id,batting_id
batting_id,,
CHA,CHW,CHA
CHN,CHC,CHN
LAA,ANA,LAA
LAN,LAD,LAN
NYA,NYY,NYA


With our abbreviation mappings, we can add new abbreviations to
the active_teams_df.

In [16]:
act_teams_mapped_df = abv_mapping_df.merge(active_teams_df, left_on='at_id', 
                                           right_index=True, how ='right')
act_teams_mapped_df.loc[list(missing_batting_teams)].head()

,at_id,batting_id,franchName,active
batting_id,,,,
FLO,FLA,FLO,Florida Marlins,Y
CHN,CHC,CHN,Chicago Cubs,Y
TBA,TBD,TBA,Tampa Bay Rays,Y
SFN,SFG,SFN,San Francisco Giants,Y
NYN,NYM,NYN,New York Mets,Y


One last issue before we put it all together. Take a look at the team names in park_factors_df and active_teams_df.

In [17]:
pf_teams = park_factors_df['Team']
at_teams = act_teams_mapped_df['franchName']

print(set(pf_teams) ^ set(at_teams))

{'Milwaukee Brewers', 'Orioles', 'Pirates', 'Minnesota Twins', 'New York Mets', 'Devil Rays', 'Seattle Mariners', 'Baltimore Orioles', 'Oakland Athletics', 'Padres', 'San Francisco Giants', 'Kansas City Royals', 'Diamondbacks', 'Chicago Cubs', 'Florida Marlins', 'Red Sox', 'Rockies', 'Angels', 'Royals', 'Mariners', 'San Diego Padres', 'Athletics', 'Rays', 'New York Yankees', 'Nationals', 'Phillies', 'White Sox', 'Blue Jays', 'Detroit Tigers', 'Los Angeles Angels of Anaheim', 'Atlanta Braves', 'Philadelphia Phillies', 'Houston Astros', 'Indians', 'Marlins', 'Mets', 'Cardinals', 'Brewers', 'Twins', 'Yankees', 'St. Louis Cardinals', 'Rangers', 'Los Angeles Dodgers', 'Arizona Diamondbacks', 'Reds', 'Tampa Bay Rays', 'Giants', 'Cleveland Indians', 'Cubs', 'Dodgers', 'Cincinnati Reds', 'Texas Rangers', 'Washington Nationals', 'Colorado Rockies', 'Tigers', 'Astros', 'Toronto Blue Jays', 'Braves', 'Boston Red Sox', 'Pittsburgh Pirates', 'Chicago White Sox'}


The team names do not match either. With a few exceptions, the last word in active_teams' team names matches the names in park_factors_df. We can start there.

In [18]:
split_teams = lambda teams: [team.split(' ')[-1] for team in teams]
pf_last, at_last = (split_teams(teams) for teams in [pf_teams, at_teams])

set(pf_teams) ^ set(at_last)

{'Anaheim',
 'Angels',
 'Blue Jays',
 'Devil Rays',
 'Jays',
 'Red Sox',
 'Sox',
 'White Sox'}

In [19]:
'Red Sox' in set(pf_teams) and 'Red Sox' in set(at_last)

False

In [20]:
'White Sox' in set(pf_teams) and 'White Sox' in set(at_last)

False

In [21]:
'Jays' in set(pf_teams) and 'Jays' in set(at_last)

False

It's easy to see what our issues are. To get the mappings to match, we can:
1. Add our shortened team names to active_teams_df
2. Fix "Sox", "Jays", and "Angels" 

In [22]:
act_teams_mapped_df['short_name'] = at_last
act_teams_mapped_df[act_teams_mapped_df['short_name']=='Sox']

,at_id,batting_id,franchName,active,short_name
batting_id,,,,,
CHA,CHW,CHA,Chicago White Sox,Y,Sox
BOS,BOS,BOS,Boston Red Sox,Y,Sox


In [23]:
indexes_to_chg = ['BOS', 'CHA', 'LAA', 'TOR']
names_to_chg = ['Red Sox', 'White Sox', 'Angels', 'Blue Jays']
act_teams_mapped_df.loc[indexes_to_chg, 'short_name'] = names_to_chg

short_names = act_teams_mapped_df['short_name']
act_teams_mapped_df.loc[indexes_to_chg]

,at_id,batting_id,franchName,active,short_name
batting_id,,,,,
BOS,BOS,BOS,Boston Red Sox,Y,Red Sox
CHA,CHW,CHA,Chicago White Sox,Y,White Sox
LAA,ANA,LAA,Los Angeles Angels of Anaheim,Y,Angels
TOR,TOR,TOR,Toronto Blue Jays,Y,Blue Jays


Now we can finally put it all together and create a unique, useful index for park_factors_df.

In [24]:
pf_mapped_df = park_factors_df.merge(act_teams_mapped_df, left_on = 'Team',
                                     right_on = 'short_name')
pf_mapped_index = get_combined_index(pf_mapped_df, 'Season', 'batting_id', left_is_num=True)
pf_mapped_df = pf_mapped_df.set_index(pf_mapped_index)
pf_mapped_df.head()

,Season,Team,Basic,1B,2B,3B,HR,SO,BB,GB,FB,LD,IFFB,FIP,at_id,batting_id,franchName,active,short_name
2008_LAA,2008,Angels,99,102,99,86,97,99,98,99,102,101,95,99,ANA,LAA,Los Angeles Angels of Anaheim,Y,Angels
2009_LAA,2009,Angels,99,101,98,85,97,99,98,100,101,100,96,99,ANA,LAA,Los Angeles Angels of Anaheim,Y,Angels
2018_LAA,2018,Angels,98,99,95,84,100,102,98,100,100,98,100,100,ANA,LAA,Los Angeles Angels of Anaheim,Y,Angels
2015_LAA,2015,Angels,97,100,96,88,98,102,97,101,100,98,100,98,ANA,LAA,Los Angeles Angels of Anaheim,Y,Angels
2014_LAA,2014,Angels,95,99,96,91,95,102,97,100,100,97,98,97,ANA,LAA,Los Angeles Angels of Anaheim,Y,Angels


We have the four dataframes we need to calculate wRC+ loaded now. However, writing the code for the calculation will be easier if we merge the dataframe together into one.

We only need select columns from each dataframe, so I will only include those values. I'll also rename some columns, so we do not have overlapping column names.

Here I remove uneeded columns from the wOBA constants dataframe:

In [25]:
cols_to_rename = {'wOBA':'year_wOBA', 'wOBAScale':'year_wOBAScale', 'wBB':'wuBB'}
wOBA_constants_df = wOBA_constants_df.rename(columns=cols_to_rename)

unneeded_cols = set(['runSB', 'runCS', 'R/W', 'cFIP'])
need_cols_set = set(wOBA_constants_df.columns.values) - unneeded_cols

wOBA_consts_short_df = wOBA_constants_df[list(need_cols_set)]
wOBA_consts_short_df.head()

,year_wOBAScale,w1B,wHR,w3B,wHBP,wuBB,R/PA,year_wOBA,w2B
Season,,,,,,,,,
2019,1.152,0.869,1.934,1.526,0.720,0.691,0.127,0.321,1.215
2018,1.226,0.880,2.031,1.578,0.720,0.690,0.117,0.315,1.247
2017,1.185,0.877,1.980,1.552,0.723,0.693,0.122,0.321,1.232
2016,1.212,0.878,2.015,1.569,0.721,0.691,0.118,0.318,1.242
2015,1.251,0.881,2.065,1.594,0.718,0.687,0.113,0.313,1.256


Next, I do the same for the league factors dataframe.

In [26]:
cols_to_rename = {'PA':'lg_PA', 'wRC':'lg_wRC'}
lg_factors_df = lg_factors_df.rename(columns=cols_to_rename)

lg_fact_short_df = lg_factors_df[['lg_PA', 'lg_wRC']]
lg_fact_short_df.head()

,lg_PA,lg_wRC
2006_NL,94664,12380
2007_NL,95178,12359
2018_NL,88080,10757
2010_NL,93061,11155
2009_NL,93840,11543


Lastly, I remove columns from the park factors dataframe.

In [27]:
cols_to_rename = {'Basic':'park_factor'}
pf_mapped_df = pf_mapped_df.rename(columns=cols_to_rename)

park_fact_short_df = pf_mapped_df[['park_factor']]
park_fact_short_df.head()

,park_factor
2008_LAA,99
2009_LAA,99
2018_LAA,98
2015_LAA,97
2014_LAA,95


In the next three code chunks, I merge the four dataframes:

In [28]:
batting_wOBA_df = batting_df.merge(wOBA_consts_short_df, left_on='yearID', 
                                   right_index=True)
batting_wOBA_df[['year_wOBA', 'year_wOBAScale']].head()

,year_wOBA,year_wOBAScale
player_season,,
florera02_2017,0.321,1.185
bryankr01_2017,0.321,1.185
hernago01_2017,0.321,1.185
josepto01_2017,0.321,1.185
confomi01_2017,0.321,1.185


In [29]:
batting_wOBA_df['year_lg'] = get_combined_index(batting_wOBA_df, 'yearID', 
                                                'lgID', left_is_num=True)

batting_lg_df = batting_wOBA_df.merge(lg_fact_short_df, left_on='year_lg', 
                                      right_index=True)

batting_lg_df[['lg_PA', 'lg_wRC']].head()

,lg_PA,lg_wRC
player_season,,
florera02_2017,92265,11385
barneda01_2017,92265,11385
pujolal01_2017,92265,11385
ramirjo01_2017,92265,11385
bettsmo01_2017,92265,11385


In [30]:
batting_lg_df['year_team'] = get_combined_index(batting_lg_df, 'yearID', 
                                                'teamID', left_is_num=True)

batting_park_df = batting_lg_df.merge(park_fact_short_df, left_on='year_team', 
                                      right_index=True)

batting_park_df[['park_factor']].head()

,park_factor
player_season,
florera02_2017,98
pujolal01_2017,98
valbulu01_2017,98
grateju01_2017,98
maybica01_2017,98


Voilà! The merged dataframe:

In [31]:
batting_park_df.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,wHBP,wuBB,R/PA,year_wOBA,w2B,year_lg,lg_PA,lg_wRC,year_team,park_factor
player_season,,,,,,,,,,,,,,,,,,,,,
florera02_2017,florera02,2017,1,LAA,AL,3,8,0,1,0,...,0.723,0.693,0.122,0.321,1.232,2017_AL,92265,11385,2017_LAA,98
pujolal01_2017,pujolal01,2017,1,LAA,AL,149,593,53,143,17,...,0.723,0.693,0.122,0.321,1.232,2017_AL,92265,11385,2017_LAA,98
valbulu01_2017,valbulu01,2017,1,LAA,AL,117,347,42,69,15,...,0.723,0.693,0.122,0.321,1.232,2017_AL,92265,11385,2017_LAA,98
grateju01_2017,grateju01,2017,1,LAA,AL,48,84,5,17,4,...,0.723,0.693,0.122,0.321,1.232,2017_AL,92265,11385,2017_LAA,98
maybica01_2017,maybica01,2017,1,LAA,AL,93,336,57,79,19,...,0.723,0.693,0.122,0.321,1.232,2017_AL,92265,11385,2017_LAA,98


Now we can start performing the wRC+ calculation! Here is the formula from Fangraphs:


<img src="https://library.fangraphs.com/wp-content/uploads/sites/5/2010/02/wRC-Flash-Card-7-19-15-e1437317254601.png">


I'm going to calculate it slowly here to show each step, starting with [wRAA](https://library.fangraphs.com/offense/wraa/). wRAA is defined as:

> wRAA = ((wOBA – league wOBA) / wOBA scale) × PA

We have two of those three values in our dataframe already, but we are missing player wOBA. [Here's the definition of wOBA from Fangraphs](https://library.fangraphs.com/offense/woba/):

> wOBA = (uBB_constant × uBB + HBP_constant × HBP + 1B_constant × 1B + 2B_constant × 2B + 3B_constant × 3B + HR_constant × HR) / (AB + BB – IBB + SF + HBP)

Still, if you are not a huge baseball fan, that might not have cleared much up for you. We can think of wOBA as the sum product of the relative difficulty of each type of batter outcome in a given run environment divided by the number of chances the player received.

Some of the stats needed for wOBA are not included expressly in our dataset, but they are easy for us to find with basic addition and subtraction.

"Unintentional Walks" (uBB) is all walks minus intentional walks:

In [32]:
batting_park_df['uBB'] = batting_park_df['BB'] - batting_park_df['IBB']

Singles (1B) is all hits minus the sum of doubles (2B), triples (3B), and home runs (HR):

In [33]:
batting_park_df['1B'] = batting_park_df['H'] - sum([batting_park_df['2B'], 
                                                    batting_park_df['3B'], 
                                                    batting_park_df['HR']])

Plate Appearences (PA) is the sum of at bats (AB), walks (BB), hit by pitches (HBP), sacrifice hits (SH), and sacrifice flies (SF).

In [34]:
batting_park_df['PA'] = sum([batting_park_df['AB'], batting_park_df['BB'], 
                             batting_park_df['HBP'], batting_park_df['SH'], 
                             batting_park_df['SF']])

batting_park_df[['uBB', '1B', 'PA']].head()

,uBB,1B,PA
player_season,,,
florera02_2017,0.0,1,9.0
pujolal01_2017,32.0,103,636.0
valbulu01_2017,47.0,32,401.0
grateju01_2017,1.0,13,87.0
maybica01_2017,47.0,53,387.0


You might have noticed that our dataframe has a column named "stint". A stint greater than 1 indicates that the player played for more than one team in a year.

In [35]:
batting_park_df[batting_park_df['stint']>1].head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,year_wOBA,w2B,year_lg,lg_PA,lg_wRC,year_team,park_factor,uBB,1B,PA
player_season,,,,,,,,,,,,,,,,,,,,,
frankni01_2017,frankni01,2017,2,LAA,AL,13,24,2,3,1,...,0.321,1.232,2017_AL,92265,11385,2017_LAA,98,5.0,2,30.0
phillbr01_2017,phillbr01,2017,2,LAA,AL,24,102,13,26,7,...,0.321,1.232,2017_AL,92265,11385,2017_LAA,98,2.0,17,105.0
uptonju01_2017,uptonju01,2017,2,LAA,AL,27,98,19,24,7,...,0.321,1.232,2017_AL,92265,11385,2017_LAA,98,16.0,10,115.0
hernate01_2017,hernate01,2017,2,TOR,AL,26,88,16,23,6,...,0.321,1.232,2017_AL,92265,11385,2017_TOR,100,6.0,9,95.0
saundmi01_2017,saundmi01,2017,2,TOR,AL,12,18,1,3,0,...,0.321,1.232,2017_AL,92265,11385,2017_TOR,100,2.0,3,20.0


That means that our index is not in fact unique. For example, the Mets traded for Yoenis Cespedes in the middle of the 2015 season (he helped them get to the World Series that year!).

In [36]:
batting_park_df.loc['cespeyo01_2015']

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,year_wOBA,w2B,year_lg,lg_PA,lg_wRC,year_team,park_factor,uBB,1B,PA
player_season,,,,,,,,,,,,,,,,,,,,,
cespeyo01_2015,cespeyo01,2015,1,DET,AL,102,403,62,118,28,...,0.313,1.256,2015_AL,91485,10593,2015_DET,102,17.0,70,427.0
cespeyo01_2015,cespeyo01,2015,2,NYN,NL,57,230,39,66,14,...,0.313,1.256,2015_NL,86736,10112,2015_NYN,95,11.0,31,249.0


I wrote a function that takes in a dataframe containing all of a player's stints, and consolidates them into a single dataframe row. The function runs a bit slow at the moment, and I plan to refactor it in the future.



My function calculates a weighted average for the player's park factor (since they played in two parks) and the player's "league scalar" (in case they moved from the National League to American or vice versa).

In [37]:
batting_park_df['league_scalar'] = batting_park_df['lg_wRC']/batting_park_df['lg_PA']

I divided my dataframe into two: one with player-seasons that appear more than once, and everyone else.

In [38]:
index = batting_park_df.index.values
dup_idx = batting_park_df.index.duplicated(keep=False)
batting_dup_df = batting_park_df.loc[set(index[dup_idx])]
batting_dup_df.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,w2B,year_lg,lg_PA,lg_wRC,year_team,park_factor,uBB,1B,PA,league_scalar
player_season,,,,,,,,,,,,,,,,,,,,,
pencehu01_2012,pencehu01,2012,2,SFN,NL,59,219,28,48,11,...,1.257,2012_NL,92469,11019,2012_SFN,93,18.0,28,248.0,0.119164
pencehu01_2012,pencehu01,2012,1,PHI,NL,101,398,59,108,15,...,1.257,2012_NL,92469,11019,2012_PHI,100,36.0,74,440.0,0.119164
bernaro01_2013,bernaro01,2013,1,WAS,NL,85,152,18,27,6,...,1.271,2013_NL,86959,9783,2013_WAS,100,11.0,18,167.0,0.112501
bernaro01_2013,bernaro01,2013,2,PHI,NL,27,75,8,14,4,...,1.271,2013_NL,86959,9783,2013_PHI,100,4.0,7,83.0,0.112501
snidetr01_2015,snidetr01,2015,1,BAL,AL,69,211,23,50,9,...,1.256,2015_AL,91485,10593,2015_BAL,101,22.0,36,236.0,0.115789


In [39]:
remaining_idx = batting_park_df.index.difference(batting_dup_df)
batting_remaining_df = batting_park_df.loc[~dup_idx]
'cespeyo01_2015' in batting_remaining_df.index.values

False

I applied my function to each repeated player-season, and then concatenated the results into a single dataframe

In [40]:
new_player_rows = [sum_player_year(batting_dup_df.loc[player_year]) 
                   for player_year in set(index[dup_idx])]

multi_team_df = pd.concat(new_player_rows, axis=1).transpose()
multi_team_df = multi_team_df.set_index(multi_team_df['player_season'])
multi_team_df.head()

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,...,w2B,w3B,wHBP,wHR,wuBB,yearID,year_lg,year_team,year_wOBA,year_wOBAScale
player_season,,,,,,,,,,,,,,,,,,,,,
pencehu01_2012,160,617,87,156,26,4,24,104,5,2,...,1.257,1.593,0.722,2.058,0.691,2012,2012_NL,2012_SFN,0.315,1.245
bernaro01_2013,112,227,26,41,10,2,4,11,4,0,...,1.271,1.616,0.722,2.101,0.69,2013,2013_NL,2013_WAS,0.314,1.277
snidetr01_2015,87,237,24,55,12,2,4,28,1,0,...,1.256,1.594,0.718,2.065,0.687,2015,2015_AL,2015_BAL,0.313,1.251
adamsma01_2017,131,339,46,93,22,1,20,65,0,0,...,1.232,1.552,0.723,1.98,0.693,2017,2017_NL,2017_SLN,0.321,1.185
shoppke01_2012,76,219,23,51,14,2,8,27,1,0,...,1.257,1.593,0.722,2.058,0.691,2012,2012_AL,2012_BOS,0.315,1.245


Lastly, I concatenated the new dataframe with the remaining players dataframe.

In [41]:
batting_no_rep_df = pd.concat([multi_team_df, batting_remaining_df], 
                              sort=False)

batting_no_rep_df.loc[['cespeyo01_2015']]

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,...,w2B,w3B,wHBP,wHR,wuBB,yearID,year_lg,year_team,year_wOBA,year_wOBAScale
player_season,,,,,,,,,,,,,,,,,,,,,
cespeyo01_2015,159,633,101,184,42,6,35,105,7,5,...,1.256,1.594,0.718,2.065,0.687,2015,2015_AL,2015_DET,0.313,1.251


We can calculate wOBA now. You might have noticed that I renamed my weight walks constant column "uBB" to "uwBB". I did that to clean up the code in my wOBA calculation below.

Each of our "hit" columns now have equivalent weighted columns, allowing me to calculate the weighted sum with a loop and some simple string concatenation.

I am using a function for the division here to fix a lower level data type issue in numpy. I am getting that issue due to my sum_player_year function that I plan to refactor.

In [42]:
numerator_cols = ['uBB', 'HBP', '1B', '2B', '3B', 'HR']

numerator_val = sum([batting_no_rep_df[col]*batting_no_rep_df['w'+col]
                     for col in numerator_cols])

denominator_val = sum([batting_no_rep_df['AB'], batting_no_rep_df['uBB'], 
                       batting_no_rep_df['SF'], batting_no_rep_df['HBP']])

batting_no_rep_df['wOBA'] = divide_w_zeros(numerator_val, denominator_val)
batting_no_rep_df[['wOBA']].head()

,wOBA
player_season,
pencehu01_2012,0.322601
bernaro01_2013,0.244964
snidetr01_2015,0.292932
adamsma01_2017,0.346141
shoppke01_2012,0.316146


Here's my wRAA calculation:

In [43]:
wRAA_scalar = batting_no_rep_df['PA'] / batting_no_rep_df['year_wOBAScale']
wOBA_above_avg = (batting_no_rep_df['wOBA'] - batting_no_rep_df['year_wOBA'])
batting_no_rep_df['wRAA'] = wOBA_above_avg * wRAA_scalar
batting_no_rep_df[['wRAA']].head()

,wRAA
player_season,
pencehu01_2012,4.20065
bernaro01_2013,-13.5154
snidetr01_2015,-4.25111
adamsma01_2017,7.78625
shoppke01_2012,0.225485


And finally wRC+:

In [44]:
wRAA_PA = divide_w_zeros(batting_no_rep_df['wRAA'], batting_no_rep_df['PA'])
lgR_PA = batting_no_rep_df['R/PA']
park_factor = batting_no_rep_df['park_factor']
lg_scalar = batting_no_rep_df['league_scalar']

wRC_PA = wRAA_PA + lgR_PA
park_R_PA_adj = lgR_PA -  lgR_PA * park_factor/100

batting_no_rep_df['wRC+'] = 100*(wRC_PA + park_R_PA_adj)/lg_scalar
batting_no_rep_df.head()

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,...,wHR,wuBB,yearID,year_lg,year_team,year_wOBA,year_wOBAScale,wOBA,wRAA,wRC+
player_season,,,,,,,,,,,,,,,,,,,,,
pencehu01_2012,160,617,87,156,26,4,24,104,5,2,...,2.058,0.691,2012,2012_NL,2012_SFN,0.315,1.245,0.322601,4.20065,103.204
bernaro01_2013,112,227,26,41,10,2,4,11,4,0,...,2.101,0.69,2013,2013_NL,2013_WAS,0.314,1.277,0.244964,-13.5154,49.7226
snidetr01_2015,87,237,24,55,12,2,4,28,1,0,...,2.065,0.687,2015,2015_AL,2015_BAL,0.313,1.251,0.292932,-4.25111,83.0187
adamsma01_2017,131,339,46,93,22,1,20,65,0,0,...,1.98,0.693,2017,2017_NL,2017_SLN,0.321,1.185,0.346141,7.78625,111.132
shoppke01_2012,76,219,23,51,14,2,8,27,1,0,...,2.058,0.691,2012,2012_AL,2012_BOS,0.315,1.245,0.316146,0.225485,96.8418


Our wRC+ calculated! In my analysis, I also use a player's wRC+ from one year prior. I put that code inside a single function that I plan to refactor which I will use here:

In [45]:
batting_t1_df = get_t1_wRC_plus(batting_no_rep_df)
batting_t1_df[['wRC+_t-1']].head()

,wRC+_t-1
player_season,
pencehu01_2012,139.365616
bernaro01_2013,113.405372
snidetr01_2015,NaN
adamsma01_2017,106.090359
shoppke01_2012,73.108598


Now it's time to import in our dependent variables. First, let's import our free agent csv's.

In [46]:
signed_df = get_df_from_dir(DIRECTORIES['signed'])
signed_df = signed_df.set_index(signed_df['Name'])
signed_df.head()

,Rk,Name,Date,To Team,From Team,Age,WAR3,Yrs,G,AB,...,WHIP,G.1,GS,SV,IP,H.1,HR.1,BB.1,SO,k
Name,,,,,,,,,,,,,,,,,,,,,
Rafael Perez\perezra01,1.0,Rafael Perez\perezra01,2013-05-23,Boston Red Sox,CLE,31,2.0,7,24.0,1.0,...,1.328,338.0,0.0,3.0,329.0,320.0,26.0,117.0,268.0,NaN
Cory Wade\wadeco01,2.0,Cory Wade\wadeco01,2013-05-13,Tampa Bay Rays,TOR,30,0.5,4,86.0,2.0,...,1.120,161.0,0.0,0.0,177.2,158.0,23.0,41.0,137.0,NaN
Josh Judy\judyjo01,3.0,Josh Judy\judyjo01,2013-05-02,Los Angeles Angels of Anaheim,CIN,27,-0.2,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Corey Patterson\patteco01,4.0,Corey Patterson\patteco01,2013-04-30,Seattle Mariners,MIL,33,0.5,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rob Delaney\delanro01,5.0,Rob Delaney\delanro01,2013-04-28,Los Angeles Angels of Anaheim,MIA,28,-0.3,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
unsigned_df = get_df_from_dir(DIRECTORIES['unsigned'])
unsigned_df = unsigned_df.set_index(unsigned_df['Name'])
unsigned_df.head()

,Rk,Name,Age,From Team,WAR3,G,PA,AB,R,H,...,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,Pos
Name,,,,,,,,,,,,,,,,,,,,,
Brandon Phillips\phillbr01,3,Brandon Phillips\phillbr01,37,LAA,5.0,1893,7965,7355,1001,2026,...,949,209,84,416,1085,0.275,0.320,0.421,0.740,2B
Yunel Escobar\escobyu01,4,Yunel Escobar\escobyu01,35,LAA,3.9,1434,5957,5315,689,1501,...,519,34,32,509,686,0.282,0.350,0.386,0.736,SS-3B
Nori Aoki\aokino01,6,Nori Aoki\aokino01,36,NYM,3.0,758,3044,2716,377,774,...,219,98,44,234,258,0.285,0.350,0.387,0.738,OF
Franklin Gutierrez\gutiefr01,8,Franklin Gutierrez\gutiefr01,35,LAD,2.6,954,3335,3032,394,778,...,361,78,19,228,736,0.257,0.311,0.408,0.719,OF-DH
Seth Smith\smithse01,9,Seth Smith\smithse01,35,BAL,2.6,1249,4084,3582,525,934,...,458,22,7,431,797,0.261,0.344,0.446,0.790,OF-DH


Right off the bat, we can see that these csv's contain a lot of data we don't need. We are only really interested if a player is in the dataframe for a given year.

In addition, the player ID we need is concatenated with the player name&mdash;we can split those up though. I'll just show the signed_df here since both signed_df and unsigned_df have almost identical formats.

In [48]:
signed_ids = signed_df['Name'].apply(lambda x: x.split('\\')[1])
signed_df = signed_df.set_index(signed_ids)
signed_df.head()

,Rk,Name,Date,To Team,From Team,Age,WAR3,Yrs,G,AB,...,WHIP,G.1,GS,SV,IP,H.1,HR.1,BB.1,SO,k
Name,,,,,,,,,,,,,,,,,,,,,
perezra01,1.0,Rafael Perez\perezra01,2013-05-23,Boston Red Sox,CLE,31,2.0,7,24.0,1.0,...,1.328,338.0,0.0,3.0,329.0,320.0,26.0,117.0,268.0,NaN
wadeco01,2.0,Cory Wade\wadeco01,2013-05-13,Tampa Bay Rays,TOR,30,0.5,4,86.0,2.0,...,1.120,161.0,0.0,0.0,177.2,158.0,23.0,41.0,137.0,NaN
judyjo01,3.0,Josh Judy\judyjo01,2013-05-02,Los Angeles Angels of Anaheim,CIN,27,-0.2,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
patteco01,4.0,Corey Patterson\patteco01,2013-04-30,Seattle Mariners,MIL,33,0.5,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
delanro01,5.0,Rob Delaney\delanro01,2013-04-28,Los Angeles Angels of Anaheim,MIA,28,-0.3,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We'll need the year as well to match out batting_df.

In [49]:
signed_df.columns.values

array(['Rk', 'Name', 'Date', 'To Team', 'From Team', 'Age', 'WAR3', 'Yrs',
       'G', 'AB', 'R', 'H', 'HR', 'RBI', 'SB', 'BB', 'BA', 'OBP', 'SLG',
       'OPS', 'W', 'L', 'ERA', 'WHIP', 'G.1', 'GS', 'SV', 'IP', 'H.1',
       'HR.1', 'BB.1', 'SO', 'k'], dtype=object)

Yikes, there's no year column! However, when I downloaded the files, I put the year in the filename:

In [50]:
print(os.listdir(DIRECTORIES['signed']))

['2012_signed.csv', '2006_signed.csv', '2017_signed.csv', '2010_signed.csv', '2018_signed.csv', '2015_signed.csv', '2009_signed.csv', '2007_signed.csv', '2016_signed.csv', '2013_signed.csv', '2014_signed.csv', '2008_signed.csv', '2011_signed.csv']


I wrote a function to handle the following:
* Loop through the file names in a directory
* Pull the year from the file name
* Fix the IDs and concatenate them with the year
* Add a named column

I choose to put this in a function since a lot of this work is relatively low level, but in a future update, I might do it interactively in the notebook.

In [51]:
signed_df = get_fa_df(DIRECTORIES['signed'], 'is_signed')
signed_df.head()

,is_signed
ids,
perezra01_2012,True
wadeco01_2012,True
judyjo01_2012,True
patteco01_2012,True
delanro01_2012,True


In [52]:
unsigned_df = get_fa_df(DIRECTORIES['unsigned'], 'is_unsigned')
unsigned_df.head()

,is_unsigned
ids,
phillbr01_2017,True
escobyu01_2017,True
aokino01_2017,True
gutiefr01_2017,True
smithse01_2017,True


In [53]:
batting_signed_df = pd.merge(batting_t1_df, signed_df, left_index=True, 
                          right_index=True, how='left')
batting_signed_df['is_signed'] = batting_signed_df['is_signed'].fillna(False)
batting_signed_df.head()

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,...,yearID,year_lg,year_team,year_wOBA,year_wOBAScale,wOBA,wRAA,wRC+,wRC+_t-1,is_signed
player_season,,,,,,,,,,,,,,,,,,,,,
pencehu01_2012,160,617,87,156,26,4,24,104,5,2,...,2012,2012_NL,2012_SFN,0.315,1.245,0.322601,4.20065,103.204,139.365616,False
bernaro01_2013,112,227,26,41,10,2,4,11,4,0,...,2013,2013_NL,2013_WAS,0.314,1.277,0.244964,-13.5154,49.7226,113.405372,True
snidetr01_2015,87,237,24,55,12,2,4,28,1,0,...,2015,2015_AL,2015_BAL,0.313,1.251,0.292932,-4.25111,83.0187,NaN,True
adamsma01_2017,131,339,46,93,22,1,20,65,0,0,...,2017,2017_NL,2017_SLN,0.321,1.185,0.346141,7.78625,111.132,106.090359,True
shoppke01_2012,76,219,23,51,14,2,8,27,1,0,...,2012,2012_AL,2012_BOS,0.315,1.245,0.316146,0.225485,96.8418,73.108598,True


In [54]:
batting_fa_df = pd.merge(batting_signed_df, unsigned_df, left_index=True, 
                      right_index=True, how='left')
batting_fa_df['is_unsigned'] = batting_fa_df['is_unsigned'].fillna(False)
batting_fa_df.head()

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,...,year_lg,year_team,year_wOBA,year_wOBAScale,wOBA,wRAA,wRC+,wRC+_t-1,is_signed,is_unsigned
player_season,,,,,,,,,,,,,,,,,,,,,
pencehu01_2012,160,617,87,156,26,4,24,104,5,2,...,2012_NL,2012_SFN,0.315,1.245,0.322601,4.20065,103.204,139.365616,False,False
bernaro01_2013,112,227,26,41,10,2,4,11,4,0,...,2013_NL,2013_WAS,0.314,1.277,0.244964,-13.5154,49.7226,113.405372,True,False
snidetr01_2015,87,237,24,55,12,2,4,28,1,0,...,2015_AL,2015_BAL,0.313,1.251,0.292932,-4.25111,83.0187,NaN,True,False
adamsma01_2017,131,339,46,93,22,1,20,65,0,0,...,2017_NL,2017_SLN,0.321,1.185,0.346141,7.78625,111.132,106.090359,True,False
shoppke01_2012,76,219,23,51,14,2,8,27,1,0,...,2012_AL,2012_BOS,0.315,1.245,0.316146,0.225485,96.8418,73.108598,True,False


Before we move on from free agent data, I am going to create an "is free agent" column to make it easier to filter free agents.

In [55]:
batting_fa_df['is_fa'] = (batting_fa_df['is_signed'] | 
                              batting_fa_df['is_unsigned'])

batting_fa_df[['is_fa']].head()

,is_fa
player_season,
pencehu01_2012,False
bernaro01_2013,True
snidetr01_2015,True
adamsma01_2017,True
shoppke01_2012,True


At this time, I have not documented my clean up of the service time data due to time constraints. In a future update I will do so, but for now, the cleaned up data is saved at [./service_time/](/service_time/). I wrote some code to interactively clean the data, which you can find [here](./service_time/service_time_cleanup.ipynb).

My csv only contains players who have reached at least 6 years of service time. I chose 6 years of service time because it's the minimum need for a player to hit free agency.

For the time being, I'll import my cleaned service time dataframe and remove the unneeded columns.

In [56]:
service_time_df = pd.read_csv(FILE_LOCS['service_time'])
service_time_index = get_combined_index(service_time_df, 'playerID', 'year', 
                                        right_is_num=True)
service_time_df = service_time_df.set_index(service_time_index)
service_time_short_df = service_time_df[['service_time']]
service_time_short_df.head()

,service_time
aardsda01_2013,7.029
aardsda01_2012,6.082
abadfe01_2017,6.073
abreubo01_2008,11.165
abreubo01_2013,15.147


I'll merge it with my batting_df too.

In [57]:
batting_st_df = batting_fa_df.merge(service_time_short_df, left_index=True, 
                                    right_index=True, how='left')
batting_st_df[['service_time']].head()

,service_time
abadan01_2006,NaN
abercre01_2006,NaN
abercre01_2007,NaN
abercre01_2008,NaN
abreubo01_2006,9.165


Time to check out the completed dataframe:

In [58]:
batting_st_df.head()

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,...,year_wOBA,year_wOBAScale,wOBA,wRAA,wRC+,wRC+_t-1,is_signed,is_unsigned,is_fa,service_time
abadan01_2006,5,3,0,0,0,0,0,0,0,0,...,0.332,1.17,0.283200,-0.208547,62.5261,NaN,True,False,True,NaN
abercre01_2006,111,255,39,54,12,2,5,24,6,5,...,0.332,1.17,0.263800,-16.3797,53.7011,NaN,False,False,False,NaN
abercre01_2007,35,76,16,15,3,0,2,5,7,1,...,0.331,1.192,0.245262,-5.75419,41.0566,53.701099,False,False,False,NaN
abercre01_2008,34,55,10,17,5,0,2,5,5,2,...,0.328,1.211,0.364220,1.79457,121.27,41.056586,True,False,True,NaN
abreubo01_2006,156,548,98,163,41,2,15,107,30,6,...,0.332,1.17,0.387537,32.5626,131.995,NaN,False,False,False,9.165


That's a lot of data! I am going to save it to csv, so we can import it in the notebook "analysis.ipynb".

In [59]:
batting_st_df.to_csv('./data/dataset.csv')